In [1]:
import pandas as pd
import numpy as np
from time import time

from cobra.io import load_model
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

from cobra.io.json import load_json_model

# cobra_model = load_model("textbook")
cobra_model = load_json_model('toy_example.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
cobra_model

Name,toy
Memory address,0x07fd435d2edf0
Number of metabolites,17
Number of reactions,21
Number of groups,0
Objective expression,1.0*OutputX - 1.0*OutputX_reverse_e7472
Compartments,


In [51]:
import itertools

rxn_names = []
rxn_indexes = []
for rxn in cobra_model.reactions:
    rxn_names.append(rxn.id)
for i in range(len(cobra_model.reactions)):
    rxn_indexes.append(i)

df = pd.DataFrame()
df['names'] = rxn_names
df['indexes'] = rxn_indexes

somelists = []
for i in range(2): somelists.append(df['indexes'])

per = []
for element in itertools.product(*somelists): 
    per.append(element)

per = pd.DataFrame(per)
# index_to_drop = per[per[0] == per[1]].index
# per.drop(index_to_drop, inplace=True)
# index_to_drop = per[per[0] == per[2]].index
# per.drop(index_to_drop, inplace=True)
# index_to_drop = per[per[1] == per[2]].index
# per.drop(index_to_drop, inplace=True)

In [71]:
df_rxn_del = pd.DataFrame()
for rxn in cobra_model.reactions:
    df_rxn_del[rxn.id] = np.nan


growth = []
status = []

for row in range(len(per)):
    for col in range(per.shape[1]):
        reaction = per.iloc[row][col]
        with cobra_model:
            cobra_model.reactions[reaction].knock_out()
    with cobra_model:
        solution = cobra_model.optimize()
        growth.append(solution.objective_value)
        status.append(solution.status)

        df_rxn_del.loc[row] = solution.fluxes

df_rxn_del['growth'] = growth
df_rxn_del['status'] = status

In [55]:
df_rxn_del = pd.DataFrame()

for rxn in cobra_model.reactions:
    df_rxn_del[rxn.id] = np.nan

rxn_names = []
growth = []
status = []

counter = 0
for rxn_a in range(len(cobra_model.reactions)):
    for rxn_b in range(rxn_a, len(cobra_model.reactions)):
        with cobra_model:
            cobra_model.reactions[rxn_a].knock_out()
            cobra_model.reactions[rxn_b].knock_out()
            solution =  cobra_model.optimize()
            rxn_names.append(cobra_model.reactions[rxn_a].id + ',' + cobra_model.reactions[rxn_b].id)
            growth.append(solution.objective_value)
            status.append(solution.status)
            
            df_rxn_del.loc[counter] = solution.fluxes
            counter = counter + 1            
            
    df_rxn_del['id'] = rxn_names
    df_rxn_del['growth'] = growth
    df_rxn_del['status'] = status

# print all infeasible knockouts
df_rxn_del

/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/comex/Desktop/python-envs/main-bio

,r1,r3,r2,r4,r5,r6,r7,r8,r9,r10,...,r14,r15,r16,InputA,InputI,OutputX,OutputN,id,growth,status
0,0.000000,1.666667e+02,0.0,0.0,333.333333,333.333333,333.333333,0.0,166.666667,1.666667e+02,...,0.0,333.333333,0.000000,0.000000,166.666667,1000.0,0.000000,"r1,r1",1000.0,optimal
1,0.000000,-2.842171e-14,0.0,0.0,333.333333,333.333333,333.333333,0.0,333.333333,-2.842171e-14,...,0.0,333.333333,333.333333,0.000000,333.333333,1000.0,333.333333,"r1,r3",1000.0,optimal
2,0.000000,-2.842171e-14,0.0,0.0,333.333333,333.333333,333.333333,0.0,333.333333,-2.842171e-14,...,0.0,333.333333,333.333333,0.000000,333.333333,1000.0,333.333333,"r1,r2",1000.0,optimal
3,0.000000,-2.842171e-14,0.0,0.0,333.333333,333.333333,333.333333,0.0,333.333333,-2.842171e-14,...,0.0,333.333333,333.333333,0.000000,333.333333,1000.0,333.333333,"r1,r4",1000.0,optimal
4,0.000000,0.000000e+00,0.0,0.0,3.333333,3.333333,3.333333,0.0,3.333333,-2.220446e-16,...,0.0,3.333333,3.333333,0.000000,3.333333,10.0,3.333333,"r1,r5",10.0,infeasible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,"InputI,OutputX",0.0,infeasible
227,166.666667,1.666667e+02,0.0,0.0,333.333333,333.333333,333.333333,0.0,0.000000,0.000000e+00,...,0.0,333.333333,166.666667,166.666667,0.000000,1000.0,0.000000,"InputI,OutputN",1000.0,optimal
228,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,"OutputX,OutputX",0.0,infeasible
229,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,"OutputX,OutputN",0.0,infeasible


In [76]:
df_rxn_del[df_rxn_del['status'] == 'optimal']

,r1,r3,r2,r4,r5,r6,r7,r8,r9,r10,...,outputK,r14,r15,r16,InputA,InputI,OutputX,OutputN,growth,status
0,-2.131628e-13,2.557954e-13,0.0,0.0,333.333333,333.333333,333.333333,-9.789700e-14,333.333333,4.752741e-13,...,1000.000000,0.0,333.333333,3.333333e+02,0.0,333.333333,1000.0,3.333333e+02,1000.0,optimal
1,-2.131628e-13,2.557954e-13,0.0,0.0,333.333333,333.333333,333.333333,-9.789700e-14,333.333333,4.752741e-13,...,1000.000000,0.0,333.333333,3.333333e+02,0.0,333.333333,1000.0,3.333333e+02,1000.0,optimal
2,-2.131628e-13,2.557954e-13,0.0,0.0,333.333333,333.333333,333.333333,-9.789700e-14,333.333333,4.752741e-13,...,1000.000000,0.0,333.333333,3.333333e+02,0.0,333.333333,1000.0,3.333333e+02,1000.0,optimal
3,-2.131628e-13,2.557954e-13,0.0,0.0,333.333333,333.333333,333.333333,-9.789700e-14,333.333333,4.752741e-13,...,1000.000000,0.0,333.333333,3.333333e+02,0.0,333.333333,1000.0,3.333333e+02,1000.0,optimal
4,-2.131628e-13,2.557954e-13,0.0,0.0,333.333333,333.333333,333.333333,-9.789700e-14,333.333333,4.752741e-13,...,1000.000000,0.0,333.333333,3.333333e+02,0.0,333.333333,1000.0,3.333333e+02,1000.0,optimal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,0.000000e+00,1.666667e+02,0.0,0.0,333.333333,333.333333,333.333333,0.000000e+00,166.666667,1.666667e+02,...,666.666667,0.0,333.333333,-1.079137e-13,0.0,166.666667,1000.0,-1.079137e-13,1000.0,optimal
437,0.000000e+00,1.666667e+02,0.0,0.0,333.333333,333.333333,333.333333,0.000000e+00,166.666667,1.666667e+02,...,666.666667,0.0,333.333333,-1.079137e-13,0.0,166.666667,1000.0,-1.079137e-13,1000.0,optimal
438,0.000000e+00,1.666667e+02,0.0,0.0,333.333333,333.333333,333.333333,0.000000e+00,166.666667,1.666667e+02,...,666.666667,0.0,333.333333,-1.079137e-13,0.0,166.666667,1000.0,-1.079137e-13,1000.0,optimal
439,0.000000e+00,5.668747e-14,0.0,0.0,333.333333,333.333333,333.333333,0.000000e+00,333.333333,7.984590e-14,...,1000.000000,0.0,333.333333,3.333333e+02,0.0,333.333333,1000.0,3.333333e+02,1000.0,optimal
